In [59]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [60]:
dataset = pd.read_csv('../Data/dataset.csv')

dataset.head(3)

,B0043GG,B0043HH,B0043II,Protein,Leukocyte,SG,Bilirubin,Acidity,Nitrite,Urobilinogen,...,Urin_color_Dk.Yellow,Urin_color_Lt. Yellow,Urin_color_Lt.Orange,Urin_color_Orange,Urin_color_Other,Urin_color_Red,Urin_color_Reddish,Urin_color_Straw,Urin_color_Yellow,Urin_color_yellow
0,9.3,251.5,4.3,1,119,1.020,0.1,5.0,103842,0.7,...,0,0,0,0,0,0,0,0,1,0
1,5.1,548.3,9.1,4,106,1.020,0.0,5.0,108098,0.7,...,0,0,0,0,0,0,0,0,1,0
2,12.2,1590.0,23.5,6,657,1.016,0.1,5.0,27515,0.7,...,0,0,0,0,0,0,0,0,1,0


In [61]:
dataset.corr()['Nitrite'].abs().sort_values(ascending=False)

Nitrite                  1.000000
Urin_color_Red           0.098163
B0043II                  0.085341
Bilirubin                0.082774
Urin_color_Reddish       0.052840
Urin_color_Orange        0.050245
Urobilinogen             0.049850
Urin_color_yellow        0.043194
Urin_color_DK.Orange     0.042346
B0043GG                  0.036130
Acidity                  0.034194
Urin_color_Dk.Yellow     0.033451
Urin_color_Lt. Yellow    0.028507
Occult_blood             0.027737
SG                       0.020128
Leukocyte                0.018149
Protein                  0.014989
B0043HH                  0.014762
Ketone_body              0.012623
Urin_color_Lt.Orange     0.010034
Urin_color_Yellow        0.007177
Urin_color_Dk. Yellow    0.006111
Urin_color_Straw         0.005212
Urin_color_Dk.Orange     0.005000
Glucose                  0.004171
Urin_color_Brown         0.002391
Urin_color_Other         0.002374
Urin_color_Dk.Red        0.002137
Urin_color_DK.Yellow     0.000475
Urin_color_Amb

In [62]:
# Selecing columns

dataset = dataset[['Urin_color_Red', 'B0043II', 'Bilirubin', 'Nitrite']]

dataset.head(3)

,Urin_color_Red,B0043II,Bilirubin,Nitrite
0,0,4.3,0.1,103842
1,0,9.1,0.0,108098
2,0,23.5,0.1,27515


In [63]:
dataset.shape

(74337, 4)

In [64]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

X.shape, y.shape

((74337, 3), (74337,))

# Train, Test Split

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

X_train.shape, X_test.shape

((59469, 3), (14868, 3))

In [13]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)

# X_train.shape, X_val.shape

((47575, 3), (11894, 3))

# Data Scaling

In [67]:
from sklearn.preprocessing import StandardScaler

In [68]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
# X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [69]:
X_train[:5, :]

array([[-0.13197082, -0.34908997,  0.19038558],
       [-0.13197082, -0.19377343, -0.54462091],
       [-0.13197082, -0.3678028 ,  0.55788882],
       [-0.13197082, -0.17880316, -0.54462091],
       [ 7.57743263, -0.30230788,  0.55788882]])

# Modeling

## 1. RandomForestRegressor

In [70]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [71]:
params = {
    'n_estimators' : range(100, 200, 5),
    'verbose' : [True]
}

In [72]:
rfr = RandomForestRegressor(n_jobs=8, random_state=2022)

In [95]:
# 0값 확인
# 0값 존재 -> MAPE 사용 불가 (실제 y값으로 나누기 때문(y > 0 필수))

len(dataset.loc[dataset.Nitrite == 0])

0

In [73]:
kfold = KFold(5, shuffle=True, random_state=2022)

### Learning

In [74]:
grid = GridSearchCV(rfr, params, n_jobs=8, cv=kfold, refit=True)
grid.fit(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 160 out of 160 | elapsed:    0.9s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=RandomForestRegressor(n_jobs=8, random_state=2022),
             n_jobs=8,
             param_grid={'n_estimators': range(100, 200, 5), 'verbose': [True]})

In [75]:
grid.best_params_

{'n_estimators': 160, 'verbose': True}

In [76]:
y_pred = grid.best_estimator_.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 160 out of 160 | elapsed:    0.0s finished


### Evaluating

In [85]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

In [78]:
mean_absolute_percentage_error(y_test, y_pred)

2.519033325163572

In [84]:
mean_absolute_error(y_test, y_pred)

11501.855077807999

In [87]:
np.sqrt(mean_squared_error(y_test, y_pred))

20345.363319817636

### Comparising y_pred, y_result

In [88]:
comparison = pd.DataFrame({'prediction' : y_pred, 'result' : y_test.values.ravel()})

In [89]:
comparison

,prediction,result
0,53072.940127,53091
1,83948.000000,83948
2,84887.000000,84887
3,53069.149764,53091
4,46895.564099,63492
...,...,...
14863,69550.500495,83948
14864,53394.371111,53091
14865,61782.441976,33303
14866,56255.664058,55594


### Saving Model

In [90]:
import pickle

In [91]:
saved_model = pickle.dump(grid.best_estimator_, open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'wb'),
                          protocol=pickle.HIGHEST_PROTOCOL)

### Loading Model

In [92]:
loaded_model = pickle.load(open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'rb'))

In [ ]:
y_pred = loaded_model.predict(X_test)

In [93]:
mean_absolute_percentage_error(y_test, y_pred)

2.519033325163572

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))